In [ ]:
'''
  Hãy xây dựng mô hình để tạo một trợ lý ảo (agent) làm nhiệm vụ chăm sóc khách hàng trong một lĩnh vực nào đó tự bạn chọn, ví dụ:
  -Agent chăm sóc, giới thiệu và trả lời câu hỏi khách hàng cho công ty du lịch


  Các nhiệm vụ cần làm:
  -Thu thập và xây dựng dữ liệu huấn luyện
  -Lựa chọn mô hình huấn luyện
  -Đánh giá độ chính xác của mô hình
'''

'\n  Hãy xây dựng mô hình để tạo một trợ lý ảo (agent) làm nhiệm vụ chăm sóc khách hàng trong một lĩnh vực nào đó tự bạn chọn, ví dụ:\n  -Agent chăm sóc, giới thiệu và trả lời câu hỏi khách hàng cho công ty du lịch\n\n\n  Các nhiệm vụ cần làm:\n  -Thu thập và xây dựng dữ liệu huấn luyện\n  -Lựa chọn mô hình huấn luyện\n  -Đánh giá độ chính xác của mô hình\n'

In [ ]:
# Dataset tự làm - lọc dữ liệu các trang báo du lịch Việt Nam

# LSTM

# Libraries

In [ ]:
import pandas as pd
import numpy as np

import string
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [ ]:
url = "/content/drive/MyDrive/datasets_task2.csv"

df = pd.read_csv(url)

print(df.shape)
print(df.head())

(606, 7)
                        Title  \
0  Tổng quan du lịch Việt Nam   
1  Tổng quan du lịch Việt Nam   
2  Tổng quan du lịch Việt Nam   
3  Tổng quan du lịch Việt Nam   
4  Tổng quan du lịch Việt Nam   

                                             context  \
0  Ngành Du lịch là ngành kinh tế tổng hợp, gồm n...   
1  Ngành Du lịch là ngành kinh tế tổng hợp, gồm n...   
2  Ngành du lịch có sự đóng góp to lớn vào tổng s...   
3  Ngành du lịch có sự đóng góp to lớn vào tổng s...   
4  Ngành du lịch có sự đóng góp to lớn vào tổng s...   

                                            question  \
0  Ngành du lịch Việt Nam có phải là ngành kinh t...   
1            Ngành du lịch gồm những ngành nghề nào?   
2                Ngành du lịch có những đóng góp gì?   
3  Ngành du lịch có giúp ích gì cho các ngành khá...   
4  Các nhóm nghề trong ngành du lịch là những ngh...   

                                             answers  is_impossible  \
0            Ngành Du lịch là ngành kinh tế tổn

In [ ]:
# Drop unnecessary columns
df = df.drop(columns=["context", "Title",
                      "is_impossible", "id", "answer_start"], axis = 1)

print(df.shape)
print(df.head())

(606, 2)
                                            question  \
0  Ngành du lịch Việt Nam có phải là ngành kinh t...   
1            Ngành du lịch gồm những ngành nghề nào?   
2                Ngành du lịch có những đóng góp gì?   
3  Ngành du lịch có giúp ích gì cho các ngành khá...   
4  Các nhóm nghề trong ngành du lịch là những ngh...   

                                             answers  
0            Ngành Du lịch là ngành kinh tế tổng hợp  
1  gồm nhiều nhóm ngành nghề liên quan đến nhiệm ...  
2  Ngành du lịch có sự đóng góp to lớn vào tổng s...  
3  Du lịch thúc đẩy hỗ trợ các ngành như giao thô...  
4  Ngành Du lịch hiện có các nhóm nghề chính như ...  


# Preprocessing Data

In [ ]:
# Check for NaN values
df.isnull().sum()

question    0
answers     2
dtype: int64

In [ ]:
# Drop rows with NaN values
df = df.dropna()

In [ ]:
# Ensure the NaN values are handled
df.isnull().sum()

question    0
answers     0
dtype: int64

In [ ]:
# Define the training data and labels
train_data = df['question'].tolist()
train_labels = df['answers'].tolist()

In [ ]:
# Encode the labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(train_labels)

# Tokenize

In [ ]:
# Tokenize the training data
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_data)
train_sequences = tokenizer.texts_to_sequences(train_data)
train_sequences = keras.preprocessing.sequence.pad_sequences(train_sequences)

In [ ]:
# Define and compile the model
model = keras.models.Sequential()

model.add(keras.layers.Embedding(len(tokenizer.word_index) + 1, 100, \
                                 input_length=train_sequences.shape[1]))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(64, activation='relu'))

model.add(keras.layers.Dense(len(train_labels), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', \
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 38, 100)           76600     
                                                                 
 flatten (Flatten)           (None, 3800)              0         
                                                                 
 dense (Dense)               (None, 64)                243264    
                                                                 
 dense_1 (Dense)             (None, 604)               39260     
                                                                 
Total params: 359124 (1.37 MB)
Trainable params: 359124 (1.37 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_sequences,\
                                                    encoded_labels,\
                                                    test_size = 0.3, \
                                                    random_state = 42)

# Train Model

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=50)

Epoch 1/50
14/14 [==============================] - 1s 7ms/step - loss: 6.4126 - accuracy: 0.0047
Epoch 2/50
14/14 [==============================] - 0s 7ms/step - loss: 6.3677 - accuracy: 0.0261
Epoch 3/50
14/14 [==============================] - 0s 7ms/step - loss: 6.2757 - accuracy: 0.0261
Epoch 4/50
14/14 [==============================] - 0s 7ms/step - loss: 6.1493 - accuracy: 0.0261
Epoch 5/50
14/14 [==============================] - 0s 7ms/step - loss: 6.0461 - accuracy: 0.0261
Epoch 6/50
14/14 [==============================] - 0s 7ms/step - loss: 5.9750 - accuracy: 0.0261
Epoch 7/50
14/14 [==============================] - 0s 7ms/step - loss: 5.9180 - accuracy: 0.0261
Epoch 8/50
14/14 [==============================] - 0s 7ms/step - loss: 5.8552 - accuracy: 0.0261
Epoch 9/50
14/14 [==============================] - 0s 7ms/step - loss: 5.7847 - accuracy: 0.0261
Epoch 10/50
14/14 [==============================] - 0s 9ms/step - loss: 5.6883 - accuracy: 0.0284
Epoch 11/50
14/14 [

In [ ]:
VOCAB_SIZE = len(tokenizer.word_index) + 1
print(f'Vocabulary size : {VOCAB_SIZE}')

Vocabulary size : 766


In [ ]:
# Generate responses
def generate_response(text):
    sequence = tokenizer.texts_to_sequences([text])
    sequence = keras.preprocessing.sequence.pad_sequences(sequence, maxlen = train_sequences.shape[1])
    prediction = model.predict(sequence)
    predicted_label = np.argmax(prediction)
    response = label_encoder.inverse_transform([predicted_label])[0]
    return response

# Test

In [ ]:
print("Welcome to the Q&A bot! Type your question or 'bye' to exit.")
print()

while True:
    user_input = input("You: ")
    if user_input.lower() == 'bye':
        print("Chatbot: Goodbye!")
        break
    else:
        # Generate response using the previously defined function
        predicted_answer = generate_response(user_input)
        print("Chatbot:", predicted_answer)
        print("-" * 100)
        print()

Welcome to the Q&A bot! Type your question or 'bye' to exit.

You: Nhiệt độ trung bình ở Sapa là bao nhiêu?
1/1 [==============================] - 0s 69ms/step
Chatbot: nhiệt độ bình trung hàng năm ở Sa Pa là 15,4oC, nhiệt độ trung bình từ 18-20oC vào tháng mùa hè, vào các tháng mùa đông 10-12oC. Nhiệt độ thấp nhất vào tháng 1 là 0oC (đặc biệt có những năm xuống tới - 3,2oC)
----------------------------------------------------------------------------------------------------

You: Cảng Cái Lân ở Hạ Long là địa điểm gì?
1/1 [==============================] - 0s 23ms/step
Chatbot: Cảng Cái Lân là nhuyễn nước sâu nhất miền Bắc, nằm ở phía Bắc thành phố Hạ Long, gần Cửa Lục. Cảng Cái Lân với vị trí thuận lợi trong vùng vịnh kín, nước sâu, suối vào ít bị sa bồi, mới nước sâu hơn – 10m và khu nước trước bến sâu – 13m. Cảng có 7 cầu mã, được phép tiếp tục nhận tàu có tải trọng trên 70.000 tấn
----------------------------------------------------------------------------------------------------



# Evaluate

In [ ]:
# Evaluate the model
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

6/6 [==============================] - 0s 4ms/step
Accuracy: 0.01098901098901099


# BLEU Score

In [ ]:
def calculate_bleu_score(y_test, y_pred, label_encoder):
    bleu_scores = []
    smooth = SmoothingFunction().method1
    for true, pred in zip(y_test, y_pred):
        reference = [label_encoder.inverse_transform([true])[0].split()]
        candidate = label_encoder.inverse_transform([pred])[0].split()
        bleu_score = sentence_bleu(reference, candidate, smoothing_function=smooth)
        bleu_scores.append(bleu_score)
    return np.mean(bleu_scores)

# Example usage (assuming y_test, y_pred, and label_encoder are defined)
average_bleu_score = calculate_bleu_score(y_test, y_pred, label_encoder)
print("Average BLEU Score:", average_bleu_score)

Average BLEU Score: 0.008809940291908298
